In [3]:
!pip install torch==1.6.0
!pip install opencv-python
!pip install torchvision==0.2.2
!pip install albumentations
!pip install tensorflow
!pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 748.8 MB 16 kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 49.5 MB 29.4 MB/s 
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 5.6 MB/s 
     |████████████████████████████████| 52 kB 3.7 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user i

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import cv2
import matplotlib.pyplot as plt
import time
import glob
from tqdm import tqdm

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# import resnest.torch as resnest_torch

from torchvision import models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from resnet import ResNet, Bottleneck

from albumentations.core.transforms_interface import DualTransform, BasicTransform
import albumentations as albu

from sklearn.model_selection import StratifiedKFold

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

## util

In [2]:
config_set = {
    'dataset': {
          'name': 'SpectrogramDataset',
          'params': {
            'img_size': 224, 
            'melspectrogram_parameters': {
              'n_mels': 128, 
              'fmin': 50, 
              'fmax': 15000, 
            }
      }
    },
    'loader': {
      'train': {
        'batch_size': 6,
        'shuffle': True,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      },
      'valid': {
        'batch_size': 2,
        'shuffle': False,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      }
    }
}
SEED=100
PERIOD = 5
SPECIES_NUM = 24
EPOCH = 50
HOP_LEN = 512
SR = 48000

In [3]:
config = config_set

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
set_seed(SEED)

In [5]:
INPUT_ROOT = Path("/home/knikaido/work/Rainforest-Connection/data")
RAW_DATA = INPUT_ROOT / "rfcx-species-audio-detection"
TRAIN_AUDIO_DIR = RAW_DATA / "train"
# TRAIN_RESAMPLED_AUDIO_DIRS = [
#   INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
# ]
TEST_AUDIO_DIR = RAW_DATA / "test"
OUTPUT_DIR = './output/'

In [6]:
pred_pathes = sorted(glob.glob(OUTPUT_DIR + '*[0-9].csv'))
pred_pathes

['./output/137_sub0.csv',
 './output/137_sub1.csv',
 './output/137_sub2.csv',
 './output/137_sub3.csv',
 './output/137_sub4.csv']

In [7]:
df_pred = np.zeros([1992, 24])
for path in pred_pathes:
    df_pred += pd.read_csv(path).iloc[:, 1:].values

In [8]:
df_pred /= 5

In [9]:
sub = pd.read_csv(str(RAW_DATA / 'sample_submission.csv'))
sub.loc[:, 's0':'s23'] = df_pred

In [10]:
sub.to_csv(OUTPUT_DIR + '137sub.csv', index=False)

In [6]:
train_gby = pd.read_pickle(RAW_DATA / "train_gby_mel.pkl")
train_gby.head()

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name
0,003bec244,[14],[1],[44.544],[2531.25],[45.1307],[5531.25],/home/knikaido/work/Rainforest-Connection/Git/...
1,006ab765f,[23],[1],[39.9615],[7235.16],[46.0452],[11283.4],/home/knikaido/work/Rainforest-Connection/Git/...
2,007f87ba2,[12],[1],[39.135999999999996],[562.5],[42.272],[3281.25],/home/knikaido/work/Rainforest-Connection/Git/...
3,0099c367b,[17],[4],[51.4206],[1464.26],[55.1996],[4565.04],/home/knikaido/work/Rainforest-Connection/Git/...
4,009b760e6,[10],[1],[50.0854],[947.461],[52.5293],[10852.7],/home/knikaido/work/Rainforest-Connection/Git/...


In [7]:
def mono_to_color(
    X: np.ndarray, mean=None, std=None,
    norm_max=None, norm_min=None, eps=1e-6
):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [8]:
def get_criterion():
    pos_weights = torch.ones(SPECIES_NUM)
    pos_weights = pos_weights * SPECIES_NUM
    loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    return loss_function

In [9]:
class LitModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        self.encoder.fc = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(1024, SPECIES_NUM)
        )
        
        self.criterion = get_criterion()
        
    def forward(self, x):
        x_out = self.encoder(x)
        return x_out
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.0001, momentum=0.9)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_pred = self.encoder(x)    
        loss = self.criterion(y_pred, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_pred = self.encoder(x)
        loss = self.criterion(y_pred, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    
    def validation_epoch_end(self, validation_step_outputs):
        mean_loss = torch.stack([x for x in validation_step_outputs]).mean()
        print('valid_epoch_loss = ', mean_loss)
        self.log('valid_epoch_loss', mean_loss, prog_bar=True, logger=True)
#         tqdm.write('Dice: \t%.3f' % mean_loss)
        return mean_loss

In [10]:
def signal_to_mel(y, sr, mel_params):
    
    len_y = len(y)
    effective_length = int(SR * PERIOD)
    
    start = 0
    end = start + effective_length
    
    images = []
    while(start < len_y):
        if(end > len_y):
            break
        y_ele = y[start:end]
        
        melspec = librosa.feature.melspectrogram(y_ele, sr=sr, **mel_params['melspectrogram_parameters'])
        melspec = librosa.power_to_db(melspec).astype(np.float32)
    
        image = mono_to_color(melspec)
        height, width, _ = image.shape
        image = cv2.resize(image, (int(width * mel_params['img_size'] / height), mel_params['img_size']))
        image = np.moveaxis(image, 2, 0)
        image = (image / 255.0).astype(np.float32)
#         image = torch.from_numpy(image).clone()
        images.append(image)
        
        start = end
        end += effective_length
        
    return np.array(images)

In [11]:
test_wav_pathes = sorted(glob.glob(str(TEST_AUDIO_DIR / '*.flac')))
len(test_wav_pathes)

1992

In [12]:
device = torch.device("cuda")

In [13]:
model_pathes = sorted(glob.glob('./output/model*'))
model_pathes

['./output/model0',
 './output/model1',
 './output/model2',
 './output/model3',
 './output/model4']

In [14]:
for i, model_path in enumerate(model_pathes):
    model = LitModule()
    model.load_state_dict(torch.load(model_path))
    model.eval().to(device)
    preds = []

    for path in tqdm(test_wav_pathes):
        y, sr = sf.read(path)
        mel_img = signal_to_mel(y, sr, config["dataset"]["params"])
        mel_img = torch.from_numpy(mel_img).clone().to(device)
        pred = model(mel_img)
        pred = nn.Softmax()(pred)
        pred = torch.mean(pred, 0)
        pred = pred.to('cpu').detach().numpy().copy()
        preds.append(pred)

    preds = np.array(preds)
    sub = pd.read_csv(str(RAW_DATA / 'sample_submission.csv'))
    sub.loc[:, 's0':'s23'] = preds
    sub.to_csv(OUTPUT_DIR + '37sub' + str(i) + '.csv', index=False)
    #     break

Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
  0%|          | 0/1992 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
100%|██████████| 1992/1992 [17:39<00:00,  1.88it/s]
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
100%|██████████| 1992/1992 [17:49<00:00,  1.86it/s]
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
100%|██████████| 1992/1992 [17:42<00:00,  1.87it/s]
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
100%|██████████| 1992/1992 [17:49<00:00,  1.86it/s]
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
100%|██████████| 1992/1992 [17:39<00:00,  1.88it/s]
